# Forecast Model Run Collections (FMRC)

There are many ways one might index weather forecast output. These different ways of constructing views of a forecast data are called "Forecast Model Run Collections" (FMRC).
- "Model Run" : a single model run.
- "Constant Offset" : all values for a given lead time.
- "Constant Forecast" : all forecasts for a given time in the future.
- "Best Estimate" : A best guess stitching together the analysis or initialization fields for past forecasts with the latest forecast.

For reference, see [this classic image](https://www.unidata.ucar.edu/presentations/caron/FmrcPoster.pdf):
![FMRC indexing schematic](./fmrc.png)


Assume that a data cube has been constructed with `forecast_reference_time` (commonly `time`) and `forecast_period` (commonly `step` or `lead`) as dimensions.

Then the more complex indexing patterns --- "Constant Forecast" or "Best Estimate" --- are achievable with numpy-style vectorized indexing.
This notebook demonstrates all 4 "FMRC" indexing patterns with a custom Xarray index.



```{note}
Some complexity arises from models like HRRR where not all runs are the same length (unlike GFS).
This complexity is handled by hardcoding in what data is available for each model: for example, with HRRR we know there are 49 `step`s available every 6 hours, and 19 `steps` otherwise.
I don't know of any standard to encode this information on disk. Please reach out if you do know.
```

## Create an example data cube

Here is a synthetic year-long dataset that mimics the HRRR forecasts.

Usually individual files describe a single `step` for a single forecast initialized at `time`. `ds` below models a datacube where such files have been concatenated along the `step` and `time` dimensions.

In [ ]:
from rolodex.forecast import (
    BestEstimate,
    ConstantForecast,
    ConstantOffset,
    ForecastIndex,
    Model,
    ModelRun,
)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

xr.set_options(keep_attrs=True, display_expand_indexes=True)

ds = xr.Dataset(attrs={"description": "Example HRRR-like dataset"})
shape = {"x": 40, "time": 365 * 24, "step": 49}
ds["foo"] = (("x", "time", "step"), np.ones(tuple(shape.values())))
ds["time"] = (
    "time",
    pd.date_range("2023-01-01", freq="h", periods=shape["time"]),
    {"standard_name": "forecast_reference_time"},
)
ds["step"] = (
    "step",
    pd.to_timedelta(np.arange(0, 49), unit="hours"),
    {"standard_name": "forecast_period"},
)

ds["foo"] = xr.where(
    # With this ordering, attrs gets preserved
    ~((ds.time.dt.hour % 6 != 0) & (ds.step.dt.total_seconds() / 3600 > 18)),
    ds.foo,
    np.nan,
)
ds

In [ ]:
(
    ds.foo.isel(x=20)
    .sel(time=slice("2023-May-01", "2023-May-03"))
    .drop_vars("step")
    .plot(x="time")
)

## Create a ForecastIndex

We _choose_ to model forecast datasets by adding a dummy "forecast" variable. This allows indexing syntax like `.sel(forecast=ConstantOffset(...))`.

An alternative design would be to skip adding the dummy variable and using an accessor to enable indexing with `ds.fcst.constant_offset(...)`. The underlying concepts and logic remains the same.

In [ ]:
ds.coords["forecast"] = 0
ds.coords["forecast"].attrs["forecast_model"] = "hrrr"

# set the new index
newds = ds.drop_indexes(["time", "step"]).set_xindex(
    ["time", "step", "forecast"], ForecastIndex, model=Model.HRRR
)
newds

## "Standard" selection

The usual indexing patterns along `time` and `step` individually still work.

In [ ]:
newds.sel(time=slice("2023-05-03", None), step=slice("2h", "12h"))

## FMRC indexing

For all cases, ForecastIndex knows how to select values so there is no missing data i.e. it only returns time stamps for which there exists a forecast.

We've told the index that this model is HRRR --- the HRRR output characteristics are hard-coded in the index code. This could be refactored a little.

Other models, like GFS, don't require any configuration.

```{note}
In the examples below, we will `assert not subset.foo.isnull().any().item()` to make sure only valid timesteps are returned after indexing
```

### `BestEstimate`

In [ ]:
subset = newds.sel(forecast=BestEstimate())
assert not subset.foo.isnull().any().item()
subset

### `ConstantForecast`

**TODO:** Adding a scalar `valid_time` for `ConstantForecast` triggers a bug.

In [ ]:
subset = newds.sel(forecast=ConstantForecast("2023-12-29"))
assert not subset.foo.isnull().any().item()
subset

### `ConstantOffset`

In [ ]:
subset = newds.sel(forecast=ConstantOffset("32h"))
assert not subset.foo.isnull().any().item()
subset

### `ModelRun`

For HRRR, the 03Z forecast only has 19 timesteps

In [ ]:
subset = newds.sel(forecast=ModelRun("2023-06-30 03:00"))
assert subset.sizes["step"] == 19
assert not subset.foo.isnull().any().item()
subset

while the 06Z forecast has 49 timesteps

In [ ]:
subset = newds.sel(forecast=ModelRun("2023-06-30 06:00"))  # 49
assert subset.sizes["step"] == 49
assert not subset.foo.isnull().any().item()
subset